In [ ]:
import re
def clean_text(line):
    line = re.sub(r'-+',' ',line)
    #line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

In [ ]:

text = 'In contrast, further deletion of the C-terminal transactivation domain in the Pax5 mutants B8 and B9 can abolish transcriptional stimulation, whereas internal deletion of the conserved octapeptide motif (OP) or the partial homeodomain (HD) of Pax5 did not have any effect (Figure 3B).'
print(clean_text(text))

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-v1.1")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
print("inputs", inputs)
print("len(inputs.input_ids[0])", len(inputs.input_ids[0]))
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
print("labels", labels)
print("len(labels[0])", len(labels[0]))
outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

In [ ]:

import numpy as np
import random
def get_random_indices(sentence):
    # Get the length of the sentence
    sentence_length = len(sentence.split())
  
    masked_idx = random.sample(range(sentence_length), 10)
    print("masked_idx", masked_idx)
    # Shuffle the indices of the sentence
    indices = np.arange(0, sentence_length-1, 1)
    np.random.shuffle(indices)

    # Select the first 10 unique indices
    random_indices = indices[:10]

    # Ensure that the selected indices are distinct
    while len(np.unique(random_indices)) < 10:
        np.random.shuffle(indices)
        random_indices = indices[:10]

    return random_indices


sentences = [
    "This is a sample sentence This is a sample sentence This is a sample sentence.",
    "Another example sentence with more words This is a sample sentence This is a sample sentence.",
    "A third sentence to demonstrate the process This is a sample sentence This is a sample sentence.",
]

for sentence in sentences:
    random_indices = get_random_indices(sentence)
    print(f"Sentence: {sentence}")
    print(f"Random Indices: {random_indices}")


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

sentences = [["This is the first sentence."], ['This is the sencond sentence.']] 
encoded_inputs = tokenizer(sentences, padding="max_length", truncation=True)

print(encoded_inputs)


In [23]:
import torch
import torch.nn.functional as F

# Example shapes
input_ids = torch.randn(32, 85)  # Assuming 32 batches, each with a sequence length of 85
logits = outputs.logits[0]  # Example logits, shape [85, 28996]
labels = torch.randint(0, 28996, (32, 85))  # Assuming labels for each position in the sequence

# Ensure that logits match the number of classes in your task
num_classes = logits.size(1)

# Transpose logits to be [sequence_length, batch_size, num_classes]
logits = logits.transpose(0, 1)

# Flatten the logits and labels for the loss calculation
logits_flat = logits.contiguous().view(-1, num_classes)
labels_flat = labels.view(-1)

# Assuming a simple sequence classification task
# You can use CrossEntropyLoss along the sequence dimension
loss_fn = torch.nn.CrossEntropyLoss()

# Calculate the loss
loss = loss_fn(logits_flat, labels_flat)

# Your training/update step here


NameError: name 'outputs' is not defined

In [ ]:
import torch

# Example tensor with size [32, 85]
original_tensor = torch.randn(32, 85)
print(original_tensor)  
# Sum along the second dimension (axis 1)
summed_tensor = torch.sum(original_tensor, dim=1)
print(summed_tensor)
# Check the size of the resulting tensor
print(summed_tensor.size())  # Should print torch.Size([32])



In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model and tokenizer
model_name = 'dmis-lab/biobert-base-cased-v1.2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Input text with a [MASK] token
text = "The [MASK] is a large mammal native to North America."

# Tokenize the input text
tokens = tokenizer(text, return_tensors='pt')

# Get the model's output logits
with torch.no_grad():
    outputs = model(**tokens)
    logits = outputs.logits

# Assuming you have a [MASK] token in the input, find its position
mask_position = tokens['input_ids'][0].tolist().index(tokenizer.mask_token_id)
print("mask_position", mask_position)
# Extract the logits for the masked position
masked_logits = logits[0, mask_position]

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(masked_logits, dim=-1)

# Get the token with the highest probability (predicted token)
predicted_token_id = torch.argmax(probabilities).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]

# Replace the [MASK] token with the predicted token in the original text
result_text = text.replace('[MASK]', predicted_token)

# Print results
print("Original Text:", text)
print("Result Text:", result_text)
print("Predicted Token:", predicted_token)
print("Probabilities:", probabilities)



In [9]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
mask = "The capital of France is [MASK]."
# inputs = tokenizer.encode_plus(mask, add_special_tokens=True,
#                                     truncation_strategy ='only_first',
#                                     max_length = 11, padding='max_length', return_tensors="pt") 
inputs = tokenizer(mask, return_tensors="pt", max_length=11, padding='max_length')
print(len(inputs.input_ids))

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt", max_length=11)["input_ids"]
print("labels", labels)
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

# [ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]

# [101, 1103, 2364, 1104, 175, 10555, 1110, 103, 119, 102] dmislab token của ngta


1
labels tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119,
           102]])


11.01

In [6]:
print(inputs)

{'input_ids': [101, 1056, 1044, 1041, 1039, 1037, 1052, 1045, 1056, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [54]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt", max_length=11, padding='max_length')
print(inputs)
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)
print(tokenizer.decode(predicted_token_id))
print("predicted token id: ", predicted_token_id)
labels = tokenizer("The capital of France is Paris.", return_tensors="pt", max_length=11, padding='max_length')["input_ids"]

print("labels: ", labels)
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

#tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110,   103,   119,   102, 0]]
#tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119, 102]])

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110,   103,   119,   102,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}
unknown
predicted token id:  tensor([3655])
labels:  tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119,
           102]])


11.01

## import library

In [1]:
from argparse import ArgumentParser
import math
from pathlib import Path
import time
import torch
import logging
import json
import random
import numpy as np
import pandas as pd
from collections import namedtuple, defaultdict
from tempfile import TemporaryDirectory
from sklearn.metrics import accuracy_score
from babel.dates import format_time
import torch.nn as nn
import torch
import sys
from scipy.special import softmax
import matplotlib.pyplot as plt
sys.path.insert(1, '../')

# sys.path.insert(1, '/content/SRLPredictionEasel')

# from bert_mlm_finetune import BertForMLMPreTraining 
from transformers import BertTokenizer, BertConfig, AdamW, get_linear_schedule_with_warmup, BertForMaskedLM 
from utils_mlm import count_num_cpu_gpu
import spacy
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter()

nlp = spacy.load("en_core_web_sm")
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2", do_lower_case=True)

MLM_IGNORE_LABEL_IDX = -1
VOCAB_SIZE = 28996 
BATCH_SIZE = 32
EPOCHS = 2
MAX_SEQ_LEN = 85
NUM_CPU = count_num_cpu_gpu()[0]

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-14 13:08:01.125343: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 13:08:03.440760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 13:08:06.716099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

## define get pos tag func

In [ ]:
def get_pos_tag(text, index):
    # Process the text with spaCy
    doc = nlp(text)

    # Check if the index is within the bounds
    if index < 0 or index >= len(doc):
       return "Index out of bounds"

    # Get the POS tag for the word at the specified index
    pos_tag = doc[index].pos_
    return pos_tag

## is POS match 

In [2]:


def is_POS_match(logits, input_ids, lm_label_ids):
    '''
    Function to check if the POS tag of the masked token in the logits is the same as the POS tag of the masked token in the original text.
    Note: This function assumes that the logits are of shape # ([85, 28996]) 
    lm_label_ids: shape (batch_size, sequence_length)
    '''
    
    origin_input_id = input_ids.clone() # Origin input id:  torch.Size([85])
   
    # Find the index of the masked token from lm_label_ids
    masked_idx = torch.where(lm_label_ids != -100)[0]
    masked_idx_input = torch.where(input_ids == tokenizer.mask_token_id)[0]
   
    origin_input_id[masked_idx_input] = lm_label_ids[masked_idx] 
    
    # get pos tag of origin text
    text = tokenizer.decode(input_ids)
    origin_text = tokenizer.decode(origin_input_id) 
    print("ORIGIN TEXT: ", origin_text)
    pos_tag_origin = get_pos_tag(origin_text, masked_idx_input)
    print("POS TAG ORIGIN: ", pos_tag_origin)
   
    # Extract the logits for the masked position
    masked_logits = logits[0, masked_idx]
    print("MASKED LOGITS: ", masked_logits) # torch.Size([28996])
    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(masked_logits, dim=-1)

    # Get the token with the highest probability (predicted token)
    predicted_token_id = torch.argmax(probabilities).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]

    # Replace the [MASK] token with the predicted token in the original text
    result_text = text.replace('[MASK]', predicted_token) 
    print("RESULT TEXT: ", result_text)
    
    # get pos tag of logits
    logits_tag = get_pos_tag(result_text, masked_idx)
    print("LOGITS TAGS: ", logits_tag)
    return pos_tag_origin == logits_tag    


## custome loss

In [5]:
import torch.nn.functional as F
def custom_loss(input_ids, logits, labels):
  
    # Cross-entropy term
    cross_entropy_term = F.cross_entropy(logits, labels, reduction='none')
    print("Cross entropy term shape: ", cross_entropy_term.shape)      ##Cross entropy term shape:  torch.Size([2720])
    # logits_shape = (32, 85, VOCAB_SIZE)
    # logits_tensor = logits.view(*logits_shape)
    
    # labels_shape = (32, 85)
    # labels_tensor = labels.view(*labels_shape)
    
    
    # Custom matching term
    matching_term_lst = []
    # for logit, input_id, label in zip(logits_tensor, input_ids, labels_tensor):
       
    matching_term = is_POS_match(logits=logits, input_ids=input_ids, lm_label_ids=labels)
    print("Matching term: ", matching_term) 
    matching_term_lst.append(matching_term) 
    # hay mình thử sửa cái POS cho cái batch luôn kh, tại cái logit với cái label truyền vô là cái batch á
    # mà t sợ nhiều khi mình reshape sai nên nó tính sai 
    matching_term = torch.tensor(matching_term_lst)
    # Combine terms
    loss = 0.5 * cross_entropy_term + (1 - matching_term)
    return loss

## train with costom loss

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")


In [3]:
### THEO CAI LOSS NAY NHA PHAT OIIIIIII
# https://discuss.huggingface.co/t/bertformaskedlm-s-loss-and-scores-how-the-loss-is-computed/607/2
import copy
from transformers import BertForMaskedLM, BertTokenizerFast
import torch
model = BertForMaskedLM.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
tokenizer = BertTokenizerFast.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

text = "Who was Jim Paterson ? Jim Paterson is a doctor".lower()
inputs  =  tokenizer.encode_plus(text,  return_tensors="pt", add_special_tokens = True, truncation=True, pad_to_max_length = True,
                                         return_attention_mask = True,  max_length=64)

input_ids = inputs['input_ids']
labels  = copy.deepcopy(input_ids) #this is the part I changed
input_ids[0][7] = tokenizer.mask_token_id
labels[input_ids != tokenizer.mask_token_id] = -100 

output = model(input_ids = input_ids, attention_mask = inputs['attention_mask'] , token_type_ids=inputs['token_type_ids'] , labels=labels)

pred = torch.argmax( output.logits[0][7]).item()
print("predicted token:", pred, tokenizer.convert_ids_to_tokens([pred])  )


# logSoftmax = torch.nn.LogSoftmax(dim=1)
# NLLLos = torch.nn.NLLLoss()

# print(NLLLos( logSoftmax(torch.unsqueeze(output.logits[0][7], 0)), torch.tensor([pred]))) #the same as F.cross_entropy(scores.view(-1, tokenizer.vocab_size), labels.view(-1))
# calculate loss manually
import torch.nn.functional as F
loss2 = F.cross_entropy(output.logits.view(-1, tokenizer.vocab_size), labels.view(-1))
print(loss2)


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


predicted token: 119 ['.']
tensor(3.5668, grad_fn=<NllLossBackward0>)


In [55]:
print(output.logits.shape)

torch.Size([1, 64, 30522])


## test loss cross entropy
